# **Métrica de Schwarzchild: más allá de los agujeros negros**

**Integrantes:**
1. Jerónimo López Gómez C.C. 10017353334
2. Valentina Lobo Ruiz C.C. 1067912559

## **Planteamiento del Problema**

Se busca analizar las propiedades de los agujeros negros, la métrica que los rige que es la métrica de Schwarzschild, su formación y su interacción con otros cuerpos celestes. Estos objetos tienen propiedades muy características y desde esta perspectiva hacen mas comprensible los fenómenos físicos que ocurren alrededor de este tipo de concentraciones de masa. En este trabajo se proponen los siguientes objetivos:

1. Repasar conceptualmente la métrica de Schwarzchild.
2. Plantear teoricamente las trayectorias que deben seguir los cuerpos y la luz en interacción con el campo gravitacional de un agujero negro.
3. Realizar la solución númerica para analizar los efectos de las concentraciones de masa en el espacio-tiempo de Schwarzschild.



In [2]:
#@title **Instalación de EinsteinPy**

#pip install einsteinpy

In [3]:
#Librerias

import numpy as np
import matplotlib.pyplot as plt
from scipy.misc import derivative
from scipy.integrate import solve_ivp
from mpl_toolkits import mplot3d

import astropy.units as u
import numpy as np
import astropy.units as u
import matplotlib.pyplot as plt

from einsteinpy.coordinates import BoyerLindquistDifferential
from einsteinpy.metric import Kerr


from einsteinpy.geodesic import Timelike
from einsteinpy.plotting.geodesic import GeodesicPlotter

## **Métrica de Schwarzild**


## **Modelado de las trayectorias de partículas**

La métrica de Schwarzild en coordenadas esfericas $(t,r,\theta,\phi)$ esta dada por: [PONER]

$$ds^2=-(1-\frac{2GM}{r})dt^2+(1-\frac{2GM}{r})^{-1}dr^2+r^2d\Omega^2$$

con $d\Omega^2=d\theta^2+sin^2\theta d\phi^2$ y M la masa del objeto gravitacional.

Considerando la ecuación de Einstein en el vacío $R_{\mu \nu}=0$ y la ausencia de energía-momento fuera del objeto, se deduce que el tensor energía-momento $T^{\alpha \beta}$ es un tensor nulo. Esto no significa que el tensor de curvatura de Riemamann sea cero, ni que el espacio-tiempo sea plano como ocurre en la métrica de Minkowski, sino que no hay cambios inmediatos en el volumen ocupado por un objeto o conjunto de partículas mientras se mueven a lo largo de geodésicas en el espacio-tiempo curvo.

Por otro lado, tenemos la ecuación de campo de Einstein está dada por: [4]

$$R_{\mu \nu} - \frac{1}{2} R g_{\mu \nu} - \Lambda g_{\mu \nu} = \frac{8 \pi G}{c^4} T_{\mu \nu}$$


De lo anterior se sigue que:

$$R_{\mu \nu} - \frac{1}{2} R g_{\mu \nu} = 0 \Leftrightarrow R_{\mu \nu} g^{\mu \nu} - \frac{1}{2} R g_{\mu \nu} g^{\mu \nu} = 0 \Leftrightarrow R^{\mu}_{\mu} - \frac{1}{2} R \delta^{\mu}_{\mu} = 0$$

$$\Leftrightarrow R - \frac{1}{2} R (4) = 0 \Leftrightarrow R = 0$$

A medida que nos alejamos del objeto masivo, los efectos gravitacionales disminuyen, llevando el espacio-tiempo a una métrica de Minkowski. Como se supone que el objeto a tratar es esférico, se debe usar la métrica para espacios-tiempos planos en coordenadas esféricas $(t,r,\theta  ,\phi )$.

$$g = \left( \begin{array}{ccc}
1 & 0 & 0 & 0 \\
0 & -1 & 0 & 0 \\
0 & 0 & -r^2 & 0 \\
0 & 0 & 0 & -r^2 \sin^2{\theta  } \\
\end{array} \right) $$

Esta será la métrica para un cuerpo lo suficientemente alejado del objeto generador del campo gravitacional.

Sin embargo, en el caso en que se está lo suficientemente cerca pero fuera del objeto productor del campo, la métrica no corresponderá a la de un espacio-tiempo plano, sino a uno curvo. Usando el límite de bajas velocidades y gravedad débil, la métrica  reproduce la gravedad newtoniana.

$$g = \left( \begin{array}{ccc}
1 - \frac{2GM}{r c^2} & 0 & 0 & 0 \\
0 & -(1 - \frac{2GM}{r c^2})^{-1} & 0 & 0 \\
0 & 0 & -r^2 & 0 \\
0 & 0 & 0 & -r^2 \sin^2{\theta  } \\
\end{array} \right) $$

Esta es la métrica encontrada por Schwarzschild que resuelve y cumple las condiciones del problema planteado. Donde $ \frac{2GM}{c^2} = r_s$ se la conoce como radio de Schwarzschild u horizonte de eventos.

Para hallar las diferentes trayectorias del sistema se debe resolver la ecuación geodésica $$\frac{d^2 x^\sigma }{d \tau^2} +  \Gamma_{\mu \nu}^{\sigma }\frac{d x^\mu}{d \tau  } \frac{d x^\nu}{d \tau} = 0$$

A continución se graficarán las trayectorias que recorren dos objetos cuando se acercan a un agujero negro.

In [ ]:
#@title Solución de las derivadas parciales
# Función cálculo derivadas parciales de función de varias variables, respecto a variable dada en punto especifico.
def partial_derivative(funcion,variable, punto,*argumentos):
    punto = punto.T
    x0 = punto[variable]

    def funcion1var(x,*argumentos): # toma una función de varias variables y la transforma en función de una variable

        X = (punto[:variable], np.array([x]), punto[(variable + 1):])

        return funcion(np.concatenate(X) ,*argumentos)

    return derivative(funcion1var, x0, args = argumentos)

# tensor métrico
def g(vector,index):
    t,r,theta,phi = vector

    if index[0] != index[1]: return 0

    else:
        if index[0]== 0 and index[1] == 0: return 1-(2*G*M/(r*c**2))
        elif index[0]== 1 and index[1] == 1: return -(1-(2*G*M/(r*c**2)))**-1
        if index[0]== 2 and index[1] == 2: return -r**2
        elif index[0]== 3 and index[1] == 3: return - (r*np.sin(theta))**2


# derivada parcial de la métrica.
def Partial_gab_c(variable,punto,index):
    Argumentos = index

    return partial_derivative(g, variable, punto, Argumentos)

# símbolos de Christoffel.
def christoffel_ikl_(punto,indices):
    i,k,l = indices

    def val(m):
        g_mk_l = Partial_gab_c(l,punto,(m,k))
        g_ml_k = Partial_gab_c(k,punto,(m,l))
        g_kl_m = Partial_gab_c(m,punto,(k,l))
        return  0.5*g(punto,(i,m))*(g_mk_l + g_ml_k + g_kl_m )

    return val(0)+val(1)+val(2)+val(3)

# lado derecho de la eq geodésica
def RightSide(S,i):
    U = S[:4]
    X = S[4:]

    return np.sum( [-christoffel_ikl_(X,(i,k,l))*U[k]*U[l]  for k in range(0,4) for l in range(0,4) ]   )

def F(t,S):
    U = S[:4]
    X = S[4:]
    f = np.array([RightSide(S,i) for i in range(0,4)])

    return np.concatenate((f,U))